# Evaluation of experiments - E1 COGNITION
fistly define the UUID of the experiment to use

### Deprecated
This approach is too slow in inference, we go for a convolutional implementation of the dense layers, please go to next experiments

In [31]:
experimentId="ca058290-1b4e-4883-94b8-537361a7d7e8"

-------------------------------------------------------

In [32]:
#external libraries
import numpy as np
import os
import matplotlib.pyplot as plt
import matplotlib.colors as clt
import plotly
import plotly.subplots as sb
import plotly.express as px
import plotly.graph_objects as go
import dotenv
import pandas as pd
import scipy.fft as fft
import scipy.signal as sg
import scipy.io as sio
import pickle as pkl
import xgboost as xgb
import time
import sklearn.metrics as skm

#project library
from spinco import *

#environment variables
dotenv.load_dotenv('lab.env')

#project variables
datapath=os.environ['DATAPATH']
cognipath=datapath+"\\COGNITION"
dreamspath=datapath+"\\DREAMS"
masspath=datapath+"\\MASS"


## load cognition

In [33]:
samplerate=200  #Should rethink this
inputDimension=601

In [34]:
signals, annotations, signalsMetadata = loadCOGNITIONSpindles(cognipath,returnSignals=True)

KeyboardInterrupt: 

In [ ]:
annotations=annotations[annotations.type=='spindle'].reset_index(drop=True)
len(annotations)

2785

In [ ]:
signalsMetadata.head()

,filename,channel,subjectId,duration,samplerate,isOriginalSamplerate,database
0,COG001_canal_C3.txt,C3-M2,0001,39600,200,False,COGNITION
1,COG002_canal_C3.txt,C3-M2,0002,39600,200,False,COGNITION
2,COG003_canal_C3.txt,C3-M2,0003,39600,200,False,COGNITION
3,COG004_canal_C3.txt,C3-M2,0004,39600,200,False,COGNITION
4,COG005_canal_C3.txt,C3-M2,0005,39600,200,False,COGNITION


## Preprocess

broadband in 0-40Hz, previous experiments (up tp 50Hz, are now in folders as .../features_old/...)

In [ ]:
def preprocessVector(vector,samplerate):
    #1. Lowpass 40Hz
    vector=filterBand(vector,[0,40],samplerate,filterOrder=4)
    #2. Z-score
    m=np.mean(vector)
    s=np.std(vector)
    vector=(vector-m)/s
    return vector

In [ ]:
for subject,signal in signals.items():
    signals[subject]=preprocessVector(signal,samplerate)

## prepare slides and labels for training

In [ ]:
signalsMetadata['excerptDimension']=signalsMetadata.apply(lambda row: int(row.duration*row.samplerate),axis=1)
allExtended=pd.DataFrame()
for ind,row in signalsMetadata.iterrows():
    thisExtended=pd.DataFrame({
        'trainIndex':np.arange(row.excerptDimension),
        'subjectId':row.subjectId
    })
    allExtended=pd.concat([allExtended,thisExtended])

In [ ]:
usedAnnotations=annotations[annotations.labelerId=='0001'].reset_index(drop=True)

In [ ]:
labels={}
for index, row in signalsMetadata.iterrows():
    labels[row.subjectId]=excerptAnnotationsToLabels(usedAnnotations[usedAnnotations.subjectId==row.subjectId],excerptDimension=row.excerptDimension)

In [ ]:
slides={}
for index, row in signalsMetadata.iterrows():
    paddedSignal=padVectorBothSides(signals[row.subjectId],window2half(inputDimension),method='closest')
    slides[row.subjectId]=np_tricks.sliding_window_view(paddedSignal,(inputDimension,))

## Load experiment results

In [ ]:
experimentModels = loadExperiment(experimentId,datapath)

In [ ]:
experimentModels.head()

,criteriumId,criteriumName,labelerIdList,train,val,test,modelId
0,0001,E1,[0001],"[0004, 0005, 0006, 0007]","[0003, 0008]",0002,04a1db9a-a55d-4c61-9aca-e9176cf39cfe
1,0001,E1,[0001],"[0002, 0005, 0006, 0008]","[0007, 0004]",0003,7660a83c-462e-480a-9c21-b9a672763ef1
2,0001,E1,[0001],"[0002, 0003, 0005, 0007]","[0006, 0008]",0004,08c6cc01-0d83-44b4-a587-9ca9a36dfb14
3,0001,E1,[0001],"[0002, 0003, 0007, 0008]","[0006, 0004]",0005,721fa0ce-6946-45a2-8332-b310eefd8481
4,0001,E1,[0001],"[0002, 0003, 0007, 0008]","[0004, 0005]",0006,c45b44b5-f5bb-4ccf-bda7-e373c18b55ef


## Optimal hyperparameter estimation with E1 criterium
we test the optimal points for the prediction threshold and number of boost iterations in the different validation groups

In [ ]:
experimentModels=experimentModels[experimentModels.criteriumName=='E1'].reset_index(drop=True)
experimentModels

,criteriumId,criteriumName,labelerIdList,train,val,test,modelId
0,0001,E1,[0001],"[0004, 0005, 0006, 0007]","[0003, 0008]",0002,04a1db9a-a55d-4c61-9aca-e9176cf39cfe
1,0001,E1,[0001],"[0002, 0005, 0006, 0008]","[0007, 0004]",0003,7660a83c-462e-480a-9c21-b9a672763ef1
2,0001,E1,[0001],"[0002, 0003, 0005, 0007]","[0006, 0008]",0004,08c6cc01-0d83-44b4-a587-9ca9a36dfb14
3,0001,E1,[0001],"[0002, 0003, 0007, 0008]","[0006, 0004]",0005,721fa0ce-6946-45a2-8332-b310eefd8481
4,0001,E1,[0001],"[0002, 0003, 0007, 0008]","[0004, 0005]",0006,c45b44b5-f5bb-4ccf-bda7-e373c18b55ef
5,0001,E1,[0001],"[0002, 0004, 0005, 0008]","[0003, 0006]",0007,1aa18dd2-aaf7-4ed0-a335-18698a433632
6,0001,E1,[0001],"[0002, 0005, 0006, 0007]","[0003, 0004]",0008,a5692535-9a66-4d75-836a-8effb7c1423c


In [ ]:
auxThres=pd.DataFrame({
    'hyperThres':[0.1,0.2,0.3,0.4,0.5,0.6]
})
auxMinDuration=pd.DataFrame({
    'hyperMinDuration':[0.3]
})
""" auxMaxDuration=pd.DataFrame({
    'hyperMaxDuration':[5]
}) """
auxClose=pd.DataFrame({
    'hyperClose':[0.1]
})


hyperParams=pd.merge(auxThres,auxMinDuration,how='cross')
""" hyperParams=pd.merge(hyperParams,auxMaxDuration,how='cross') """
hyperParams=pd.merge(hyperParams,auxClose,how='cross')

hyperParams

,hyperThres,hyperMinDuration,hyperClose
0,0.1,0.3,0.1
1,0.2,0.3,0.1
2,0.3,0.3,0.1
3,0.4,0.3,0.1
4,0.5,0.3,0.1
5,0.6,0.3,0.1


In [ ]:
hyperExperimentModels=[]
hyperMeanF1=[]
hyperStdF1=[]

for ind_hyper,row_hyper in hyperParams.iterrows():
    print(ind_hyper)
    thisExperimentModels=experimentModels.copy()
    hyperThres=row_hyper.hyperThres
    hyperMinDuration=row_hyper.hyperMinDuration
    hyperClose=row_hyper.hyperClose
    
    meanF1=[]
    meanPrecision=[]
    meanRecall=[]

    stdF1=[]
    stdPrecision=[]
    stdRecall=[]

    for ind,row in thisExperimentModels.iterrows():
        #load model
        model=loadDL(row.modelId,experimentId,datapath)
        #initialise lists
        f1s=[]
        precisions=[]
        recalls=[]

        #iterate validation subjects
        for valSubjectId in row.val:
            inferred=model.predict(np.asarray(slides[valSubjectId]),verbose=0)
            rawLabels=inferred>=hyperThres
            processedLabels=labelingProcess(rawLabels.flatten(),hyperClose,hyperMinDuration,samplerate)
            processedAnnotations=labelVectorToAnnotations(processedLabels,samplerate)
            gtAnnotations=labelVectorToAnnotations(labels[valSubjectId],samplerate) 
            f,r,p=annotationPairToMetrics(gtAnnotations,processedAnnotations)

            #Metric appends
            f1s.append(f)
            precisions.append(p)
            recalls.append(r)

        #statistics of the metrics over the subjects of the validation set
        meanF1.append(np.mean(f1s))
        meanPrecision.append(np.mean(precisions))
        meanRecall.append(np.mean(recalls))

        stdF1.append(np.std(f1s))
        stdPrecision.append(np.std(precisions))
        stdRecall.append(np.std(recalls))
        
    thisExperimentModels['meanF1']=meanF1
    thisExperimentModels['meanPrecision']=meanPrecision
    thisExperimentModels['meanRecall']=meanRecall
    
    thisExperimentModels['stdF1']=stdF1
    thisExperimentModels['stdPrecision']=stdPrecision
    thisExperimentModels['stdRecall']=stdRecall

    hyperExperimentModels.append(thisExperimentModels)
    hyperMeanF1.append(np.mean(thisExperimentModels['meanF1']))
    hyperStdF1.append(np.std(thisExperimentModels['meanF1']))
    

0
Keras model archive loading:
File Name                                             Modified             Size
config.json                                    2023-09-01 10:25:58         4112
metadata.json                                  2023-09-01 10:25:58           64
variables.h5                                   2023-09-01 10:25:58      1799176
Keras weights file (<HDF5 file "variables.h5" (mode r)>) loading:
...layers\dense
......vars
.........0
.........1
...layers\dense_1
......vars
.........0
.........1
...layers\dense_2
......vars
.........0
.........1
...layers\dense_3
......vars
.........0
.........1
...layers\dense_4
......vars
.........0
.........1
...layers\dense_5
......vars
.........0
.........1
...layers\input_layer
......vars
...metrics\mean
......vars
.........0
.........1
...optimizer
......vars
.........0
.........1
.........10
.........11
.........12
.........13
.........14
.........15
.........16
.........17
.........18
.........19
.........2
.........20
........

KeyboardInterrupt: 

In [ ]:
hyperParams['meanMeanF1']=hyperMeanF1
hyperParams['stdMeanF1']=hyperStdF1

In [ ]:
optimalInd=hyperParams[hyperParams.meanMeanF1==np.max(hyperParams.meanMeanF1)].index[0]
print("maximal mean score at:")
optimal=hyperParams.iloc[optimalInd]
optimal

maximal mean score at:


hyperThres           0.400000
hyperDepth          10.000000
hyperMinDuration     0.300000
hyperMaxDuration     5.000000
hyperClose           0.100000
meanMeanF1           0.717276
stdMeanF1            0.059869
Name: 19, dtype: float64

## Graphical representation of the optimal point

In [ ]:
aux=hyperParams[(hyperParams.hyperDepth==optimal.hyperDepth)&(hyperParams.hyperMinDuration==optimal.hyperMinDuration)&
                (hyperParams.hyperClose==optimal.hyperClose)&(hyperParams.hyperMaxDuration==optimal.hyperMaxDuration)].reset_index(drop=True)
px.scatter(aux,x='hyperThres',y='meanMeanF1',error_y='stdMeanF1')

In [ ]:
aux=hyperParams[(hyperParams.hyperThres==optimal.hyperThres)&(hyperParams.hyperMinDuration==optimal.hyperMinDuration)&
                (hyperParams.hyperClose==optimal.hyperClose)&(hyperParams.hyperMaxDuration==optimal.hyperMaxDuration)].reset_index(drop=True)
px.scatter(aux,x='hyperDepth',y='meanMeanF1',error_y='stdMeanF1')